<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/334_SalesEnab_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outreach Generation Utilities for Sales Enablement Orchestrator

In [ ]:
"""Outreach Generation Utilities for Sales Enablement Orchestrator

This module contains utilities for generating outreach recommendations.
MVP: Rule-based message generation (LLM enhancement in Phase 8).
"""

from typing import Dict, List, Any, Optional
from datetime import datetime


def select_rep_for_lead(
    lead: Dict[str, Any],
    sales_reps: List[Dict[str, Any]],
    deals_lookup: Dict[str, Dict[str, Any]]
) -> Optional[str]:
    """
    Select appropriate sales rep for a lead.

    Simple rule-based assignment:
    - If lead already has a deal, use that rep
    - Otherwise, assign based on region match
    - If no region match, assign to rep with lowest active_deals

    Args:
        lead: Lead dictionary
        sales_reps: List of sales rep dictionaries
        deals_lookup: Dictionary mapping lead_id to deal dict

    Returns:
        rep_id or None
    """
    lead_id = lead.get("lead_id")
    lead_region = lead.get("region", "North America")

    # Check if lead already has a deal with assigned rep
    deal = deals_lookup.get(lead_id)
    if deal:
        rep_id = deal.get("rep_id")
        if rep_id:
            return rep_id

    # Find reps matching region
    matching_reps = [r for r in sales_reps if r.get("region") == lead_region]

    if matching_reps:
        # Assign to rep with lowest active_deals
        matching_reps.sort(key=lambda r: r.get("active_deals", 0))
        return matching_reps[0].get("rep_id")

    # Fallback: assign to rep with lowest active_deals
    if sales_reps:
        sales_reps.sort(key=lambda r: r.get("active_deals", 0))
        return sales_reps[0].get("rep_id")

    return None


def determine_channel(
    lead: Dict[str, Any],
    interactions: Optional[List[Dict[str, Any]]] = None
) -> str:
    """
    Determine best outreach channel for a lead.

    Rule-based:
    - If recent email interaction, use email
    - If recent call, use call
    - Default: email

    Args:
        lead: Lead dictionary
        interactions: Optional list of interactions

    Returns:
        Channel: "email" | "call" | "linkedin"
    """
    if not interactions:
        return "email"

    # Check most recent interaction
    recent_interactions = sorted(
        interactions,
        key=lambda x: x.get("datetime", ""),
        reverse=True
    )

    if recent_interactions:
        last_type = recent_interactions[0].get("type", "")
        if last_type == "call":
            return "call"
        elif last_type == "email":
            return "email"

    return "email"


def generate_message_draft(
    lead: Dict[str, Any],
    customer_needs: Optional[Dict[str, Any]] = None,
    recommended_action: Optional[str] = None
) -> str:
    """
    Generate rule-based message draft for outreach.

    MVP: Template-based message (LLM enhancement in Phase 8).

    Args:
        lead: Lead dictionary
        customer_needs: Optional customer needs analysis
        recommended_action: Optional recommended action from signals

    Returns:
        Message draft string
    """
    company_name = lead.get("company_name", "there")
    persona = lead.get("persona", "decision maker")
    pain_points = lead.get("pain_points", [])
    primary_pain = pain_points[0] if pain_points else "your business challenges"

    # Build message based on recommended action
    if recommended_action:
        if "demo" in recommended_action.lower():
            action_text = "schedule a demo"
        elif "pricing" in recommended_action.lower():
            action_text = "discuss pricing options"
        elif "case study" in recommended_action.lower():
            action_text = "share a relevant case study"
        else:
            action_text = recommended_action.lower()
    else:
        action_text = "discuss how we can help"

    # Generate template message
    message = f"""Hi {persona} at {company_name},

I noticed that {company_name} is facing challenges with {primary_pain}.

Our solution has helped similar companies in {lead.get('industry', 'your industry')} address these exact pain points and achieve significant improvements.

Would you be open to {action_text}? I'd be happy to show you how we can help {company_name} overcome {primary_pain}.

Best regards"""

    return message


def determine_timing(
    lead: Dict[str, Any],
    interactions: Optional[List[Dict[str, Any]]] = None
) -> str:
    """
    Determine optimal timing for outreach.

    Rule-based:
    - If no interactions: "Tuesday 10am EST"
    - If recent interaction: "Follow up within 24-48 hours"
    - Default: "Tuesday 10am EST"

    Args:
        lead: Lead dictionary
        interactions: Optional list of interactions

    Returns:
        Timing recommendation string
    """
    if not interactions:
        return "Tuesday 10am EST"

    # Check if there's a recent interaction
    from agents.sales_enablement.utilities.follow_up_tracking import days_since

    recent_interactions = sorted(
        interactions,
        key=lambda x: x.get("datetime", ""),
        reverse=True
    )

    if recent_interactions:
        last_interaction_dt = recent_interactions[0].get("datetime")
        if last_interaction_dt:
            days = days_since(last_interaction_dt)
            if days < 2:
                return "Follow up within 24-48 hours"
            elif days < 7:
                return "This week"

    return "Tuesday 10am EST"


def generate_outreach_recommendation(
    lead: Dict[str, Any],
    sales_reps: List[Dict[str, Any]],
    interactions: Optional[List[Dict[str, Any]]] = None,
    customer_needs: Optional[Dict[str, Any]] = None,
    deals_lookup: Dict[str, Dict[str, Any]] = None,
    signals_lookup: Dict[str, Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Generate complete outreach recommendation for a lead.

    Args:
        lead: Lead dictionary
        sales_reps: List of sales rep dictionaries
        interactions: Optional list of interactions for this lead
        customer_needs: Optional customer needs analysis
        deals_lookup: Dictionary mapping lead_id to deal dict
        signals_lookup: Dictionary mapping lead_id to signal dict

    Returns:
        Outreach recommendation dictionary
    """
    lead_id = lead.get("lead_id")
    interactions = interactions or []
    deals_lookup = deals_lookup or {}
    signals_lookup = signals_lookup or {}

    # Select rep
    rep_id = select_rep_for_lead(lead, sales_reps, deals_lookup)

    # Determine channel
    channel = determine_channel(lead, interactions)

    # Get recommended action
    signal = signals_lookup.get(lead_id)
    recommended_action = signal.get("recommended_action") if signal else None

    # Generate message draft
    message_draft = generate_message_draft(lead, customer_needs, recommended_action)

    # Determine timing
    timing = determine_timing(lead, interactions)

    # Extract personalization elements
    personalization_elements = []
    if lead.get("company_name"):
        personalization_elements.append("company_name")
    if lead.get("pain_points"):
        personalization_elements.append("pain_points")
    if lead.get("industry"):
        personalization_elements.append("industry")
    if customer_needs and customer_needs.get("buying_signals"):
        personalization_elements.append("buying_signals")

    # Determine urgency
    urgency = signal.get("urgency", "medium") if signal else "medium"

    return {
        "lead_id": lead_id,
        "rep_id": rep_id,
        "channel": channel,
        "message_draft": message_draft,
        "personalization_elements": personalization_elements,
        "timing": timing,
        "next_step": recommended_action or "initiate outreach",
        "urgency": urgency
    }


def generate_all_outreach_recommendations(
    prioritized_leads: List[Dict[str, Any]],
    sales_reps: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    customer_needs_analysis: List[Dict[str, Any]],
    deals_lookup: Dict[str, Dict[str, Any]],
    signals_lookup: Dict[str, Dict[str, Any]],
    top_n: int = 10
) -> List[Dict[str, Any]]:
    """
    Generate outreach recommendations for top N prioritized leads.

    Args:
        prioritized_leads: List of prioritized leads (already sorted)
        sales_reps: List of sales rep dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        customer_needs_analysis: List of customer needs analysis dictionaries
        deals_lookup: Dictionary mapping lead_id to deal dict
        signals_lookup: Dictionary mapping lead_id to signal dict
        top_n: Number of top leads to generate outreach for

    Returns:
        List of outreach recommendation dictionaries
    """
    # Create customer needs lookup
    needs_lookup = {n["lead_id"]: n for n in customer_needs_analysis}

    recommendations = []
    top_leads = prioritized_leads[:top_n]

    for lead in top_leads:
        lead_id = lead.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        customer_needs = needs_lookup.get(lead_id)

        recommendation = generate_outreach_recommendation(
            lead,
            sales_reps,
            interactions,
            customer_needs,
            deals_lookup,
            signals_lookup
        )

        recommendations.append(recommendation)

    return recommendations



# Historical Insights Utilities for Sales Enablement Orchestrator

In [ ]:
"""Historical Insights Utilities for Sales Enablement Orchestrator

This module contains utilities for analyzing win/loss patterns from past deals.
"""

from typing import Dict, List, Any, Optional
from collections import Counter, defaultdict


def analyze_won_deals(
    deals: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]]
) -> List[Dict[str, Any]]:
    """
    Analyze won deals to identify winning patterns.

    Args:
        deals: List of all deal dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions

    Returns:
        List of win pattern dictionaries
    """
    won_deals = [d for d in deals if d.get("status") == "won"]

    if not won_deals:
        return []

    patterns = []

    # Pattern 1: Interaction frequency
    interaction_counts = []
    for deal in won_deals:
        lead_id = deal.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        interaction_counts.append(len(interactions))

    if interaction_counts:
        avg_interactions = sum(interaction_counts) / len(interaction_counts)
        min_interactions = min(interaction_counts)
        max_interactions = max(interaction_counts)

        patterns.append({
            "pattern_type": "interaction_frequency",
            "description": f"Won deals had {avg_interactions:.1f} interactions on average (range: {min_interactions}-{max_interactions})",
            "frequency": 1.0,  # All won deals
            "recommendation": f"Aim for {int(avg_interactions)}+ interactions for similar deals"
        })

    # Pattern 2: Positive sentiment
    positive_sentiment_count = 0
    for deal in won_deals:
        lead_id = deal.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        positive_interactions = [i for i in interactions if i.get("sentiment") == "positive"]
        if len(positive_interactions) >= 2:
            positive_sentiment_count += 1

    if won_deals:
        positive_frequency = positive_sentiment_count / len(won_deals)
        patterns.append({
            "pattern_type": "positive_sentiment",
            "description": f"{positive_sentiment_count}/{len(won_deals)} won deals had 2+ positive interactions",
            "frequency": positive_frequency,
            "recommendation": "Focus on building positive engagement early in the sales cycle"
        })

    # Pattern 3: Pricing discussion
    pricing_discussed_count = 0
    for deal in won_deals:
        lead_id = deal.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        for interaction in interactions:
            key_topics = interaction.get("key_topics", [])
            if "pricing" in key_topics:
                pricing_discussed_count += 1
                break

    if won_deals:
        pricing_frequency = pricing_discussed_count / len(won_deals)
        patterns.append({
            "pattern_type": "pricing_discussion",
            "description": f"{pricing_discussed_count}/{len(won_deals)} won deals discussed pricing",
            "frequency": pricing_frequency,
            "recommendation": "Introduce pricing discussion early for qualified leads"
        })

    # Pattern 4: Deal velocity (days to close)
    days_to_close = []
    for deal in won_deals:
        created_date = deal.get("created_date")
        expected_close_date = deal.get("expected_close_date")
        if created_date and expected_close_date:
            try:
                from datetime import datetime
                created = datetime.fromisoformat(created_date)
                closed = datetime.fromisoformat(expected_close_date)
                days = (closed - created).days
                if days > 0:
                    days_to_close.append(days)
            except (ValueError, AttributeError):
                pass

    if days_to_close:
        avg_days = sum(days_to_close) / len(days_to_close)
        patterns.append({
            "pattern_type": "deal_velocity",
            "description": f"Average days to close: {avg_days:.0f} days",
            "frequency": 1.0,
            "recommendation": f"Target closing similar deals within {int(avg_days)} days"
        })

    return patterns


def analyze_lost_deals(
    deals: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]]
) -> List[Dict[str, Any]]:
    """
    Analyze lost deals to identify loss patterns.

    Args:
        deals: List of all deal dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions

    Returns:
        List of loss pattern dictionaries
    """
    lost_deals = [d for d in deals if d.get("status") == "lost"]

    if not lost_deals:
        return []

    patterns = []

    # Pattern 1: Negative sentiment
    negative_sentiment_count = 0
    for deal in lost_deals:
        lead_id = deal.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        negative_interactions = [i for i in interactions if i.get("sentiment") == "negative"]
        if negative_interactions:
            negative_sentiment_count += 1

    if lost_deals:
        negative_frequency = negative_sentiment_count / len(lost_deals)
        patterns.append({
            "pattern_type": "negative_sentiment",
            "description": f"{negative_sentiment_count}/{len(lost_deals)} lost deals had negative sentiment",
            "frequency": negative_frequency,
            "recommendation": "Address negative sentiment immediately when detected"
        })

    # Pattern 2: Risk flags
    risk_flag_counts = Counter()
    for deal in lost_deals:
        risk_flags = deal.get("risk_flags", [])
        for flag in risk_flags:
            risk_flag_counts[flag] += 1

    if risk_flag_counts:
        most_common_flag = risk_flag_counts.most_common(1)[0]
        patterns.append({
            "pattern_type": "risk_flags",
            "description": f"Most common risk flag in lost deals: '{most_common_flag[0]}' ({most_common_flag[1]} occurrences)",
            "frequency": most_common_flag[1] / len(lost_deals),
            "recommendation": f"Monitor and address '{most_common_flag[0]}' risk flags proactively"
        })

    # Pattern 3: Competitors
    competitor_counts = Counter()
    for deal in lost_deals:
        competitors = deal.get("competitors", [])
        for competitor in competitors:
            competitor_counts[competitor] += 1

    if competitor_counts:
        most_common_competitor = competitor_counts.most_common(1)[0]
        patterns.append({
            "pattern_type": "competition",
            "description": f"Most common competitor in lost deals: '{most_common_competitor[0]}' ({most_common_competitor[1]} occurrences)",
            "frequency": most_common_competitor[1] / len(lost_deals),
            "recommendation": f"Develop competitive differentiation strategy against '{most_common_competitor[0]}'"
        })

    # Pattern 4: Low interaction frequency
    interaction_counts = []
    for deal in lost_deals:
        lead_id = deal.get("lead_id")
        interactions = interactions_lookup.get(lead_id, [])
        interaction_counts.append(len(interactions))

    if interaction_counts:
        avg_interactions = sum(interaction_counts) / len(interaction_counts)
        patterns.append({
            "pattern_type": "low_engagement",
            "description": f"Lost deals had {avg_interactions:.1f} interactions on average",
            "frequency": 1.0,
            "recommendation": f"Increase engagement frequency - lost deals had fewer interactions than won deals"
        })

    return patterns



# Test Phase 5: Outreach Generation & Historical Insights Utilities

In [ ]:
"""Test Phase 5: Outreach Generation & Historical Insights Utilities

Test utilities independently before building nodes.
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent
sys.path.insert(0, str(project_root))

from agents.sales_enablement.utilities.outreach_generation import (
    generate_outreach_recommendation,
    generate_all_outreach_recommendations
)
from agents.sales_enablement.utilities.historical_insights import (
    analyze_won_deals,
    analyze_lost_deals
)
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_sales_reps,
    load_deals,
    load_interactions,
    load_signals,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from config import SalesEnablementOrchestratorConfig


def test_generate_outreach_recommendation():
    """Test generating outreach recommendation"""
    print("Testing generate_outreach_recommendation...")

    config = SalesEnablementOrchestratorConfig()
    leads = load_leads(config.data_dir, config.leads_file)
    reps = load_sales_reps(config.data_dir, config.sales_reps_file)
    interactions = load_interactions(config.data_dir, config.interactions_file)
    deals = load_deals(config.data_dir, config.deals_file)
    signals = load_signals(config.data_dir, config.signals_file)

    interactions_lookup = build_interactions_lookup(interactions)
    deals_lookup = build_deals_lookup(deals)
    signals_lookup = build_signals_lookup(signals)

    lead = leads[0]
    lead_interactions = interactions_lookup.get(lead["lead_id"], [])

    recommendation = generate_outreach_recommendation(
        lead,
        reps,
        lead_interactions,
        None,  # customer_needs
        deals_lookup,
        signals_lookup
    )

    assert "lead_id" in recommendation, "should have lead_id"
    assert "rep_id" in recommendation, "should have rep_id"
    assert "channel" in recommendation, "should have channel"
    assert "message_draft" in recommendation, "should have message_draft"
    assert "timing" in recommendation, "should have timing"

    print(f"✅ generate_outreach_recommendation test passed!")
    print(f"   Lead: {recommendation['lead_id']}")
    print(f"   Rep: {recommendation['rep_id']}")
    print(f"   Channel: {recommendation['channel']}")
    print(f"   Timing: {recommendation['timing']}")
    print(f"   Message preview: {recommendation['message_draft'][:60]}...")
    print()


def test_analyze_won_deals():
    """Test analyzing won deals"""
    print("Testing analyze_won_deals...")

    config = SalesEnablementOrchestratorConfig()
    deals = load_deals(config.data_dir, config.deals_file)
    interactions = load_interactions(config.data_dir, config.interactions_file)
    interactions_lookup = build_interactions_lookup(interactions)

    win_patterns = analyze_won_deals(deals, interactions_lookup)

    assert isinstance(win_patterns, list), "should return list"
    assert len(win_patterns) > 0, "should find some patterns"
    assert "pattern_type" in win_patterns[0], "should have pattern_type"
    assert "description" in win_patterns[0], "should have description"
    assert "recommendation" in win_patterns[0], "should have recommendation"

    print(f"✅ analyze_won_deals test passed!")
    print(f"   Found {len(win_patterns)} win patterns")
    for pattern in win_patterns[:2]:
        print(f"     - {pattern['pattern_type']}: {pattern['description']}")
    print()


def test_analyze_lost_deals():
    """Test analyzing lost deals"""
    print("Testing analyze_lost_deals...")

    config = SalesEnablementOrchestratorConfig()
    deals = load_deals(config.data_dir, config.deals_file)
    interactions = load_interactions(config.data_dir, config.interactions_file)
    interactions_lookup = build_interactions_lookup(interactions)

    loss_patterns = analyze_lost_deals(deals, interactions_lookup)

    assert isinstance(loss_patterns, list), "should return list"
    if len(loss_patterns) > 0:
        assert "pattern_type" in loss_patterns[0], "should have pattern_type"
        assert "description" in loss_patterns[0], "should have description"

    print(f"✅ analyze_lost_deals test passed!")
    print(f"   Found {len(loss_patterns)} loss patterns")
    for pattern in loss_patterns[:2]:
        print(f"     - {pattern['pattern_type']}: {pattern['description']}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 5: Outreach Generation & Historical Insights - Test Suite")
    print("=" * 60)
    print()

    test_generate_outreach_recommendation()
    test_analyze_won_deals()
    test_analyze_lost_deals()

    print("=" * 60)
    print("✅ All Phase 5 utility tests passed!")
    print("=" * 60)



# Sales Enablement Orchestrator Nodes

In [ ]:
"""Sales Enablement Orchestrator Nodes

This module contains all nodes for the Sales Enablement Orchestrator workflow.
Following the MVP-first approach: all nodes are rule-based (no LLM) for Phase 1-7.
"""

import json
from typing import Dict, Any
from config import SalesEnablementOrchestratorState, SalesEnablementOrchestratorConfig
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_sales_reps,
    load_interactions,
    load_deals,
    load_signals,
    build_leads_lookup,
    build_reps_lookup,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from agents.sales_enablement.utilities.lead_prioritization import (
    prioritize_leads,
    get_top_priority_leads
)
from agents.sales_enablement.utilities.customer_analysis import (
    analyze_all_customer_needs
)
from agents.sales_enablement.utilities.follow_up_tracking import (
    extract_follow_up_actions
)
from agents.sales_enablement.utilities.rep_nudging import (
    generate_all_rep_nudges
)
from agents.sales_enablement.utilities.deal_analysis import (
    detect_stalled_deals,
    detect_at_risk_deals,
    generate_all_deal_insights
)
from agents.sales_enablement.utilities.outreach_generation import (
    generate_all_outreach_recommendations
)
from agents.sales_enablement.utilities.historical_insights import (
    analyze_won_deals,
    analyze_lost_deals
)


def goal_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for the Sales Enablement Orchestrator.

    This is a simple rule-based goal definition that sets the framework.
    No dependencies - this is the entry point.
    """
    lead_id = state.get("lead_id")
    rep_id = state.get("rep_id")
    focus_area = state.get("focus_area")

    # Build focus areas based on input
    if focus_area:
        focus_areas = [focus_area]
    else:
        focus_areas = [
            "lead_prioritization",
            "customer_needs_analysis",
            "outreach_generation",
            "follow_up_coordination",
            "rep_nudging",
            "deal_insights",
            "historical_insights",
            "executive_reporting"
        ]

    goal = {
        "objective": "Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights",
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_areas": focus_areas,
        "scope": "all_leads" if not lead_id else "single_lead",
        "scope_reps": "all_reps" if not rep_id else "single_rep"
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    Only depends on goal_node.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    focus_areas = goal.get("focus_areas", [])

    # Build plan based on focus areas
    plan = []
    step_num = 1

    # Always start with data loading
    plan.append({
        "step": step_num,
        "name": "data_loading",
        "description": "Load all data files (leads, reps, interactions, deals, signals) and create lookup dictionaries",
        "dependencies": [],
        "outputs": ["leads", "sales_reps", "interactions", "deals", "signals", "leads_lookup", "reps_lookup", "interactions_lookup", "deals_lookup", "signals_lookup"]
    })
    step_num += 1

    # Lead prioritization (if in focus areas)
    if "lead_prioritization" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "lead_prioritization",
            "description": "Rank and prioritize leads based on intent, engagement, risk, and urgency",
            "dependencies": ["data_loading"],
            "outputs": ["prioritized_leads", "top_priority_leads"]
        })
        step_num += 1

    # Customer needs analysis (if in focus areas)
    if "customer_needs_analysis" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "customer_needs_analysis",
            "description": "Analyze customer pain points, buying signals, and objection likelihood",
            "dependencies": ["data_loading"],
            "outputs": ["customer_needs_analysis"]
        })
        step_num += 1

    # Outreach generation (if in focus areas)
    if "outreach_generation" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "outreach_generation",
            "description": "Generate personalized outreach recommendations with message drafts and timing",
            "dependencies": ["lead_prioritization", "customer_needs_analysis"],
            "outputs": ["outreach_recommendations"]
        })
        step_num += 1

    # Follow-up coordination (if in focus areas)
    if "follow_up_coordination" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "follow_up_coordination",
            "description": "Identify and track follow-up actions, detect overdue items",
            "dependencies": ["data_loading"],
            "outputs": ["follow_up_actions"]
        })
        step_num += 1

    # Rep nudging (if in focus areas)
    if "rep_nudging" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "rep_nudging",
            "description": "Generate nudges for sales reps based on overdue follow-ups, stalled deals, and high-priority leads",
            "dependencies": ["follow_up_coordination", "lead_prioritization", "deal_insights"],
            "outputs": ["rep_nudges"]
        })
        step_num += 1

    # Deal insights (if in focus areas)
    if "deal_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "deal_insights",
            "description": "Detect stalled deals, at-risk deals, and opportunities",
            "dependencies": ["data_loading"],
            "outputs": ["deal_insights", "stalled_deals", "at_risk_deals"]
        })
        step_num += 1

    # Historical insights (if in focus areas)
    if "historical_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "historical_insights",
            "description": "Analyze win/loss patterns from past deals to surface actionable insights",
            "dependencies": ["data_loading"],
            "outputs": ["win_patterns", "loss_patterns"]
        })
        step_num += 1

    # Executive reporting (if in focus areas)
    if "executive_reporting" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "executive_reporting",
            "description": "Generate pipeline summary and rep performance metrics",
            "dependencies": ["deal_insights", "lead_prioritization"],
            "outputs": ["pipeline_summary", "rep_performance_summary"]
        })
        step_num += 1

    # Always end with report generation
    plan.append({
        "step": step_num,
        "name": "report_generation",
        "description": "Generate final markdown report with all insights and recommendations",
        "dependencies": [p["name"] for p in plan if p["name"] != "report_generation"],
        "outputs": ["enablement_report", "report_file_path"]
    })

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading all data files and creating lookups.

    This node loads:
    - Leads
    - Sales Reps
    - Interactions
    - Deals
    - Signals

    And creates lookup dictionaries for fast access.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Load all data files
        leads = load_leads(config.data_dir, config.leads_file)
        sales_reps = load_sales_reps(config.data_dir, config.sales_reps_file)
        interactions = load_interactions(config.data_dir, config.interactions_file)
        deals = load_deals(config.data_dir, config.deals_file)
        signals = load_signals(config.data_dir, config.signals_file)

        # Build lookup dictionaries
        leads_lookup = build_leads_lookup(leads)
        reps_lookup = build_reps_lookup(sales_reps)
        interactions_lookup = build_interactions_lookup(interactions)
        deals_lookup = build_deals_lookup(deals)
        signals_lookup = build_signals_lookup(signals)

        return {
            "leads": leads,
            "sales_reps": sales_reps,
            "interactions": interactions,
            "deals": deals,
            "signals": signals,
            "leads_lookup": leads_lookup,
            "reps_lookup": reps_lookup,
            "interactions_lookup": interactions_lookup,
            "deals_lookup": deals_lookup,
            "signals_lookup": signals_lookup,
            "errors": errors
        }
    except FileNotFoundError as e:
        return {
            "errors": errors + [f"data_loading_node: File not found - {str(e)}"]
        }
    except json.JSONDecodeError as e:
        return {
            "errors": errors + [f"data_loading_node: Invalid JSON - {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: Unexpected error - {str(e)}"]
        }


def lead_prioritization_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Lead Prioritization Node: Rank and prioritize leads.

    Calculates priority scores for all leads and selects top N.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not leads:
        return {
            "errors": errors + ["lead_prioritization_node: leads required"]
        }

    try:
        # Prioritize all leads
        prioritized_leads = prioritize_leads(
            leads,
            signals_lookup,
            weights=config.priority_scoring_weights
        )

        # Get top N leads
        top_priority_leads = get_top_priority_leads(
            prioritized_leads,
            top_n=config.top_n_leads
        )

        return {
            "prioritized_leads": prioritized_leads,
            "top_priority_leads": top_priority_leads,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"lead_prioritization_node: {str(e)}"]
        }


def customer_needs_analysis_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Customer Needs Analysis Node: Analyze customer needs and pain points.

    Analyzes pain points, buying signals, objection likelihood, and product fit.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    interactions_lookup = state.get("interactions_lookup", {})
    deals_lookup = state.get("deals_lookup", {})

    if not leads:
        return {
            "errors": errors + ["customer_needs_analysis_node: leads required"]
        }

    try:
        # Analyze all customer needs
        customer_needs_analysis = analyze_all_customer_needs(
            leads,
            interactions_lookup,
            deals_lookup
        )

        return {
            "customer_needs_analysis": customer_needs_analysis,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"customer_needs_analysis_node: {str(e)}"]
        }


def follow_up_coordination_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Follow-up Coordination Node: Identify and track follow-up actions.

    Extracts follow-up actions from interactions and tracks their status.
    """
    errors = state.get("errors", [])
    interactions = state.get("interactions", [])
    config = SalesEnablementOrchestratorConfig()

    if not interactions:
        return {
            "errors": errors + ["follow_up_coordination_node: interactions required"]
        }

    try:
        # Extract follow-up actions
        follow_up_actions = extract_follow_up_actions(
            interactions,
            follow_up_overdue_days=config.follow_up_overdue_days
        )

        return {
            "follow_up_actions": follow_up_actions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"follow_up_coordination_node: {str(e)}"]
        }


def deal_insights_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Deal Insights Node: Detect stalled deals, at-risk deals, and opportunities.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    interactions_lookup = state.get("interactions_lookup", {})
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not deals:
        return {
            "errors": errors + ["deal_insights_node: deals required"]
        }

    try:
        # Generate all deal insights
        deal_insights = generate_all_deal_insights(
            deals,
            interactions_lookup,
            signals_lookup,
            config
        )

        # Separate by type
        stalled_deals = [i for i in deal_insights if i.get("insight_type") == "stalled"]
        at_risk_deals = [i for i in deal_insights if i.get("insight_type") == "at_risk"]

        return {
            "deal_insights": deal_insights,
            "stalled_deals": stalled_deals,
            "at_risk_deals": at_risk_deals,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"deal_insights_node: {str(e)}"]
        }


def rep_nudging_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Rep Nudging Node: Generate nudges for sales reps.

    Generates nudges based on:
    - Overdue follow-ups
    - Stalled deals
    - High-priority leads with no interaction
    - Deals at risk
    """
    errors = state.get("errors", [])
    follow_up_actions = state.get("follow_up_actions", [])
    stalled_deals = state.get("stalled_deals", [])
    at_risk_deals = state.get("at_risk_deals", [])
    top_priority_leads = state.get("top_priority_leads", [])
    leads_lookup = state.get("leads_lookup", {})
    interactions_lookup = state.get("interactions_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    try:
        # Generate all nudges
        rep_nudges = generate_all_rep_nudges(
            follow_up_actions,
            stalled_deals,
            at_risk_deals,
            top_priority_leads,
            leads_lookup,
            interactions_lookup,
            config
        )

        return {
            "rep_nudges": rep_nudges,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"rep_nudging_node: {str(e)}"]
        }


def outreach_generation_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Outreach Generation Node: Generate personalized outreach recommendations.

    Creates outreach recommendations with message drafts, timing, and channel selection.
    """
    errors = state.get("errors", [])
    prioritized_leads = state.get("prioritized_leads", [])
    sales_reps = state.get("sales_reps", [])
    interactions_lookup = state.get("interactions_lookup", {})
    customer_needs_analysis = state.get("customer_needs_analysis", [])
    deals_lookup = state.get("deals_lookup", {})
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not prioritized_leads:
        return {
            "errors": errors + ["outreach_generation_node: prioritized_leads required"]
        }

    try:
        # Generate outreach recommendations for top N leads
        outreach_recommendations = generate_all_outreach_recommendations(
            prioritized_leads,
            sales_reps,
            interactions_lookup,
            customer_needs_analysis,
            deals_lookup,
            signals_lookup,
            top_n=config.top_n_leads
        )

        return {
            "outreach_recommendations": outreach_recommendations,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"outreach_generation_node: {str(e)}"]
        }


def historical_insights_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Historical Insights Node: Analyze win/loss patterns from past deals.

    Identifies patterns from won and lost deals to surface actionable insights.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    interactions_lookup = state.get("interactions_lookup", {})

    if not deals:
        return {
            "errors": errors + ["historical_insights_node: deals required"]
        }

    try:
        # Analyze won deals
        win_patterns = analyze_won_deals(deals, interactions_lookup)

        # Analyze lost deals
        loss_patterns = analyze_lost_deals(deals, interactions_lookup)

        return {
            "win_patterns": win_patterns,
            "loss_patterns": loss_patterns,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"historical_insights_node: {str(e)}"]
        }



# Test Phase 1: Foundation Nodes (Goal & Planning)

In [ ]:
"""Test Phase 1: Foundation Nodes (Goal & Planning)

Test goal_node and planning_node independently.
"""

from agents.sales_enablement.nodes import goal_node, planning_node
from config import SalesEnablementOrchestratorState


def test_goal_node():
    """Test goal_node with no input"""
    print("Testing goal_node...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result, "goal_node should return 'goal'"
    assert result["goal"]["objective"] is not None, "goal should have objective"
    assert len(result["goal"]["focus_areas"]) > 0, "goal should have focus_areas"
    assert result["goal"]["scope"] == "all_leads", "scope should be 'all_leads' when no lead_id"

    print(f"✅ goal_node test passed!")
    print(f"   Objective: {result['goal']['objective'][:60]}...")
    print(f"   Focus areas: {len(result['goal']['focus_areas'])} areas")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_lead_id():
    """Test goal_node with specific lead_id"""
    print("Testing goal_node with lead_id...")

    state: SalesEnablementOrchestratorState = {
        "lead_id": "L-001",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["lead_id"] == "L-001", "goal should include lead_id"
    assert result["goal"]["scope"] == "single_lead", "scope should be 'single_lead'"

    print(f"✅ goal_node with lead_id test passed!")
    print(f"   Lead ID: {result['goal']['lead_id']}")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_focus_area():
    """Test goal_node with specific focus_area"""
    print("Testing goal_node with focus_area...")

    state: SalesEnablementOrchestratorState = {
        "focus_area": "lead_prioritization",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["focus_areas"] == ["lead_prioritization"], "focus_areas should match input"

    print(f"✅ goal_node with focus_area test passed!")
    print(f"   Focus areas: {result['goal']['focus_areas']}")
    print()


def test_planning_node():
    """Test planning_node with goal"""
    print("Testing planning_node...")

    state: SalesEnablementOrchestratorState = {
        "goal": {
            "objective": "Test objective",
            "focus_areas": ["lead_prioritization", "outreach_generation"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result, "planning_node should return 'plan'"
    assert len(result["plan"]) > 0, "plan should have steps"

    # Check that plan includes data_loading (always first)
    assert result["plan"][0]["name"] == "data_loading", "First step should be data_loading"

    # Check that plan includes requested focus areas
    plan_names = [step["name"] for step in result["plan"]]
    assert "lead_prioritization" in plan_names, "plan should include lead_prioritization"
    assert "outreach_generation" in plan_names, "plan should include outreach_generation"

    # Check that plan ends with report_generation
    assert result["plan"][-1]["name"] == "report_generation", "Last step should be report_generation"

    print(f"✅ planning_node test passed!")
    print(f"   Total steps: {len(result['plan'])}")
    print(f"   Steps: {[step['name'] for step in result['plan']]}")
    print()


def test_planning_node_without_goal():
    """Test planning_node error handling when goal is missing"""
    print("Testing planning_node error handling...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result, "planning_node should return errors"
    assert len(result["errors"]) > 0, "should have error when goal is missing"
    assert "goal is required" in result["errors"][0], "error should mention goal"

    print(f"✅ planning_node error handling test passed!")
    print(f"   Error: {result['errors'][0]}")
    print()


def test_goal_and_planning_chain():
    """Test goal_node → planning_node chain"""
    print("Testing goal_node → planning_node chain...")

    # Start with empty state
    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    # Run goal_node (merge result into state)
    goal_result = goal_node(state)
    state.update(goal_result)
    assert "goal" in state, "goal should be in state after goal_node"

    # Run planning_node (merge result into state)
    plan_result = planning_node(state)
    state.update(plan_result)
    assert "plan" in state, "plan should be in state after planning_node"
    assert len(state["errors"]) == 0, "should have no errors in successful chain"

    print(f"✅ goal_node → planning_node chain test passed!")
    print(f"   Goal objective: {state['goal']['objective'][:50]}...")
    print(f"   Plan steps: {len(state['plan'])}")
    print()


def test_data_loading_node():
    """Test data_loading_node"""
    print("Testing data_loading_node...")

    from agents.sales_enablement.nodes import data_loading_node

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = data_loading_node(state)

    assert "leads" in result, "data_loading_node should return 'leads'"
    assert "sales_reps" in result, "data_loading_node should return 'sales_reps'"
    assert "interactions" in result, "data_loading_node should return 'interactions'"
    assert "deals" in result, "data_loading_node should return 'deals'"
    assert "signals" in result, "data_loading_node should return 'signals'"
    assert "leads_lookup" in result, "data_loading_node should return 'leads_lookup'"
    assert "reps_lookup" in result, "data_loading_node should return 'reps_lookup'"
    assert "interactions_lookup" in result, "data_loading_node should return 'interactions_lookup'"
    assert "deals_lookup" in result, "data_loading_node should return 'deals_lookup'"
    assert "signals_lookup" in result, "data_loading_node should return 'signals_lookup'"
    assert len(result["errors"]) == 0, "should have no errors"

    # Verify data loaded correctly
    assert len(result["leads"]) == 20, "should load 20 leads"
    assert len(result["sales_reps"]) == 4, "should load 4 reps"
    assert len(result["interactions"]) == 12, "should load 12 interactions"
    assert len(result["deals"]) == 15, "should load 15 deals"
    assert len(result["signals"]) == 20, "should load 20 signals"

    # Verify lookups work
    assert "L-001" in result["leads_lookup"], "leads_lookup should contain L-001"
    assert "SR-01" in result["reps_lookup"], "reps_lookup should contain SR-01"

    print(f"✅ data_loading_node test passed!")
    print(f"   Loaded: {len(result['leads'])} leads, {len(result['sales_reps'])} reps, {len(result['interactions'])} interactions, {len(result['deals'])} deals, {len(result['signals'])} signals")
    print(f"   Lookups: {len(result['leads_lookup'])} leads, {len(result['reps_lookup'])} reps, {len(result['interactions_lookup'])} leads with interactions, {len(result['deals_lookup'])} leads with deals")
    print()

    return result


def test_lead_prioritization_node():
    """Test lead_prioritization_node"""
    print("Testing lead_prioritization_node...")

    from agents.sales_enablement.nodes import lead_prioritization_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then prioritize
    result = lead_prioritization_node(state)

    assert "prioritized_leads" in result, "should return prioritized_leads"
    assert "top_priority_leads" in result, "should return top_priority_leads"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["prioritized_leads"]) == 20, "should prioritize all 20 leads"
    assert len(result["top_priority_leads"]) == 10, "should return top 10 leads"
    assert result["prioritized_leads"][0]["priority_score"] >= result["prioritized_leads"][-1]["priority_score"], "should be sorted"

    print(f"✅ lead_prioritization_node test passed!")
    print(f"   Prioritized: {len(result['prioritized_leads'])} leads")
    print(f"   Top {len(result['top_priority_leads'])} leads:")
    for i, lead in enumerate(result["top_priority_leads"][:3], 1):
        print(f"     {i}. {lead['lead_id']} - {lead['company_name']} (Score: {lead['priority_score']:.2f})")
    print()


def test_customer_needs_analysis_node():
    """Test customer_needs_analysis_node"""
    print("Testing customer_needs_analysis_node...")

    from agents.sales_enablement.nodes import customer_needs_analysis_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze
    result = customer_needs_analysis_node(state)

    assert "customer_needs_analysis" in result, "should return customer_needs_analysis"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["customer_needs_analysis"]) == 20, "should analyze all 20 leads"
    assert "buying_signals" in result["customer_needs_analysis"][0], "should have buying_signals"
    assert "product_fit" in result["customer_needs_analysis"][0], "should have product_fit"

    print(f"✅ customer_needs_analysis_node test passed!")
    print(f"   Analyzed: {len(result['customer_needs_analysis'])} leads")
    sample = result["customer_needs_analysis"][0]
    print(f"   Sample: {sample['lead_id']} - Product fit: {sample['product_fit']}, Buying signals: {len(sample['buying_signals'])}")
    print()


def test_follow_up_coordination_node():
    """Test follow_up_coordination_node"""
    print("Testing follow_up_coordination_node...")

    from agents.sales_enablement.nodes import follow_up_coordination_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then coordinate follow-ups
    result = follow_up_coordination_node(state)

    assert "follow_up_actions" in result, "should return follow_up_actions"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["follow_up_actions"]) > 0, "should find some follow-ups"
    assert "status" in result["follow_up_actions"][0], "should have status"

    print(f"✅ follow_up_coordination_node test passed!")
    print(f"   Follow-up actions: {len(result['follow_up_actions'])}")
    overdue = [f for f in result["follow_up_actions"] if f.get("status") == "overdue"]
    print(f"   Overdue: {len(overdue)}")
    print()


def test_deal_insights_node():
    """Test deal_insights_node"""
    print("Testing deal_insights_node...")

    from agents.sales_enablement.nodes import deal_insights_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze deals
    result = deal_insights_node(state)

    assert "deal_insights" in result, "should return deal_insights"
    assert "stalled_deals" in result, "should return stalled_deals"
    assert "at_risk_deals" in result, "should return at_risk_deals"
    assert len(result["errors"]) == 0, "should have no errors"

    print(f"✅ deal_insights_node test passed!")
    print(f"   Total insights: {len(result['deal_insights'])}")
    print(f"   Stalled deals: {len(result['stalled_deals'])}")
    print(f"   At-risk deals: {len(result['at_risk_deals'])}")
    print()


def test_rep_nudging_node():
    """Test rep_nudging_node"""
    print("Testing rep_nudging_node...")

    from agents.sales_enablement.nodes import (
        rep_nudging_node, data_loading_node, lead_prioritization_node,
        follow_up_coordination_node, deal_insights_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(follow_up_coordination_node(state))
    state.update(deal_insights_node(state))

    # Then generate nudges
    result = rep_nudging_node(state)

    assert "rep_nudges" in result, "should return rep_nudges"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["rep_nudges"]) > 0, "should generate some nudges"
    assert "nudge_type" in result["rep_nudges"][0], "should have nudge_type"
    assert "message" in result["rep_nudges"][0], "should have message"

    print(f"✅ rep_nudging_node test passed!")
    print(f"   Generated {len(result['rep_nudges'])} nudges")

    # Show nudge types
    nudge_types = {}
    for nudge in result["rep_nudges"]:
        nudge_type = nudge.get("nudge_type", "unknown")
        nudge_types[nudge_type] = nudge_types.get(nudge_type, 0) + 1

    print(f"   Nudge types: {nudge_types}")
    if result["rep_nudges"]:
        print(f"   Sample nudge: {result['rep_nudges'][0]['message'][:60]}...")
    print()


def test_outreach_generation_node():
    """Test outreach_generation_node"""
    print("Testing outreach_generation_node...")

    from agents.sales_enablement.nodes import (
        outreach_generation_node, data_loading_node, lead_prioritization_node,
        customer_needs_analysis_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(customer_needs_analysis_node(state))

    # Then generate outreach
    result = outreach_generation_node(state)

    assert "outreach_recommendations" in result, "should return outreach_recommendations"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["outreach_recommendations"]) > 0, "should generate some recommendations"
    assert "message_draft" in result["outreach_recommendations"][0], "should have message_draft"
    assert "channel" in result["outreach_recommendations"][0], "should have channel"

    print(f"✅ outreach_generation_node test passed!")
    print(f"   Generated {len(result['outreach_recommendations'])} outreach recommendations")
    sample = result["outreach_recommendations"][0]
    print(f"   Sample: {sample['lead_id']} -> {sample['rep_id']} via {sample['channel']}")
    print(f"   Message preview: {sample['message_draft'][:50]}...")
    print()


def test_historical_insights_node():
    """Test historical_insights_node"""
    print("Testing historical_insights_node...")

    from agents.sales_enablement.nodes import historical_insights_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze historical patterns
    result = historical_insights_node(state)

    assert "win_patterns" in result, "should return win_patterns"
    assert "loss_patterns" in result, "should return loss_patterns"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["win_patterns"]) > 0, "should find some win patterns"

    print(f"✅ historical_insights_node test passed!")
    print(f"   Win patterns: {len(result['win_patterns'])}")
    print(f"   Loss patterns: {len(result['loss_patterns'])}")
    if result["win_patterns"]:
        print(f"   Sample win pattern: {result['win_patterns'][0]['pattern_type']} - {result['win_patterns'][0]['description'][:50]}...")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 1-5: All Nodes - Test Suite")
    print("=" * 60)
    print()

    test_goal_node()
    test_goal_node_with_lead_id()
    test_goal_node_with_focus_area()
    test_planning_node()
    test_planning_node_without_goal()
    test_goal_and_planning_chain()
    test_data_loading_node()
    test_lead_prioritization_node()
    test_customer_needs_analysis_node()
    test_follow_up_coordination_node()
    test_deal_insights_node()
    test_rep_nudging_node()
    test_outreach_generation_node()
    test_historical_insights_node()

    print("=" * 60)
    print("✅ All Phase 1-5 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python test_sales_enablement_phase1.py
============================================================
Phase 1-5: All Nodes - Test Suite
============================================================

Testing goal_node...
✅ goal_node test passed!
   Objective: Enable sales team performance by prioritizing leads, analyzi...
   Focus areas: 8 areas
   Scope: all_leads

Testing goal_node with lead_id...
✅ goal_node with lead_id test passed!
   Lead ID: L-001
   Scope: single_lead

Testing goal_node with focus_area...
✅ goal_node with focus_area test passed!
   Focus areas: ['lead_prioritization']

Testing planning_node...
✅ planning_node test passed!
   Total steps: 4
   Steps: ['data_loading', 'lead_prioritization', 'outreach_generation', 'report_generation']

Testing planning_node error handling...
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

Testing goal_node → planning_node chain...
✅ goal_node → planning_node chain test passed!
   Goal objective: Enable sales team performance by prioritizing lead...
   Plan steps: 10

Testing data_loading_node...
✅ data_loading_node test passed!
   Loaded: 20 leads, 4 reps, 12 interactions, 15 deals, 20 signals
   Lookups: 20 leads, 4 reps, 9 leads with interactions, 15 leads with deals

Testing lead_prioritization_node...
✅ lead_prioritization_node test passed!
   Prioritized: 20 leads
   Top 10 leads:
     1. L-020 - Orion Aerospace (Score: 86.60)
     2. L-003 - Apex Manufacturing (Score: 85.50)
     3. L-011 - Atlas Freight (Score: 83.00)

Testing customer_needs_analysis_node...
✅ customer_needs_analysis_node test passed!
   Analyzed: 20 leads
   Sample: L-001 - Product fit: 0.63, Buying signals: 1

Testing follow_up_coordination_node...
✅ follow_up_coordination_node test passed!
   Follow-up actions: 7
   Overdue: 7

Testing deal_insights_node...
✅ deal_insights_node test passed!
   Total insights: 11
   Stalled deals: 2
   At-risk deals: 5

Testing rep_nudging_node...
✅ rep_nudging_node test passed!
   Generated 24 nudges
   Nudge types: {'follow_up_due': 7, 'stalled_deal': 2, 'high_priority_lead': 10, 'deal_at_risk': 5}
   Sample nudge: ⚠️ Follow-up with Northstar Logistics (L-001) is overdue. Yo...

Testing outreach_generation_node...
✅ outreach_generation_node test passed!
   Generated 10 outreach recommendations
   Sample: L-020 -> SR-04 via email
   Message preview: Hi VP Strategy at Orion Aerospace,

I noticed that...

Testing historical_insights_node...
✅ historical_insights_node test passed!
   Win patterns: 4
   Loss patterns: 4
   Sample win pattern: interaction_frequency - Won deals had 1.0 interactions on average (range: ...

============================================================
✅ All Phase 1-5 tests passed!
============================================================
